# Deep Learning: Introducción con TFLearn
## Sentiment analysis con TFLearn

Vamos a construir una Red Neuronal para aplicar Sentiment Analysis en review de peliculas. Vamos a utilizar [TFLearn](http://tflearn.org/), que es una librería de alto nivel contruida por encima de TensorFlow. TFLearn vuelve simple el contruir Redes Neuronales ya que solo se definen los layers. Se encarga de la mayoria de detalles por uno.


Comenzamos importando todos los módulos que vamos a utilzar, luego cargaremos la data y la prepararemos.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical

## Preparamos la data

Nuestro objetivo aquí es convertir los reviews en un vector de palabras. El vector de palabras tendrá elementos representantes del vocabulario total. Por ejemplo si la segunda posición del vector representa la palabra 'the', para cada review contaremos el número de veces que aparece 'the' y colocaremos ese valor en el vector. Les mostraré ejemplos conforme construimos el Input Data de las reviews.

### Leer u obtener la data

Usaremos la librería pandas para leer los reviews y sus etiquetas (labels) positiva/negativa de un archivo coma separado.  Los datos que estamos utilizando ya han sido preprocesados un poco y sabemos que sólo utiliza caracteres en minúsculas. Si tubieras datos sin pre procesar aquí abría que añadir un paso de limpiar los datos. Con ello solucionamos diferentes variaciones de una misma palabra por ejemplo 'the', 'The', 'tHe' y 'THE'.

In [ ]:
reviews = pd.read_csv('reviews.txt', header=None)
labels = pd.read_csv('labels.txt', header=None)

### Conteo de la frecuencia de las  palabras

Para comenzar debemos contar la frecuencia con que cada palabra aparece en los datos. Usaremos este recuento para crear un vocabulario que usaremos para codificar las reviews. A ese resultado se le llama Saco de Palabras ([bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model)). Lo utilizaremos para seleccionar el vocabulario y construir nuestro vector de palabras. Trataremos de implementarlo utilizando un [Counter class](https://docs.python.org/2/library/collections.html#collections.Counter).

> **Ejercicio:** Crear un bag of words de las reviews y asignarlo a `total_counts`. Las reviews se almacenan en el `reviews` [Pandas DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html). Si deseas las reviews como un array de Numpy, utiliza `reviews.values`. Puedes iterar las filas en el DataFrame con `for idx, row in reviews.iterrows():` ([documentación](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.iterrows.html)). Cuando dividas las reviews en palabras, usa `.split(' ')` y no `.split()` para que nos salga el mismo resultado.

In [ ]:
from collections import Counter

total_counts = # bag of words here

print("Total words in data set: ", len(total_counts))

Mantengamos las 10000 primeras palabras más frecuentes. La mayoría de las palabras del vocabulario se usan raramente por que tendrán poco efecto en nuestras predicciones. A continuación, ordenaremos `vocab` por conteo y solo usaremos las 10000 más frecuentes.

In [ ]:
vocab = sorted(total_counts, key=total_counts.get, reverse=True)[:10000]
print(vocab[:60])

¿Cuál es la última palabra en nuestro vocabulario? Podemos usar esto para juzgar si 10000 es demasiado pequeño. Si la última palabra es bastante común, es probable que tengamos que guardar más palabras.

In [ ]:
print(vocab[-1], ': ', total_counts[vocab[-1]])

La última palabra en nuestro vocabulario se muestra en 30 reviews de 25000. Creo que es justo decir que esto es una pequeña proporción de comentarios. Probablemente estamos bien con este número de palabras.

**Nota:** Cuando lo ejecutes, puede que veas una palabra diferente, pero sigue siendo de valor `30`. Eso es porque hay varias palabras con la misma cantidad de ocurrencias, y la `Counter` class no garantiza cual va a retornar en ese caso.

Ahora para cada review, crearemos un vector de palabras. Primero necesitamos hacer un mapeo de palabras a un índice, bastante fácil de hacer con una comprensión del diccionario.

> **Ejercicio:** Crear un dictionario llamado `word2idx` que asigne cada palabra del vocabulario un índice. La primera palabra en `vocab` tiene el índice `0`, la segunda palabra tiene índice `1`, y así sucesivamente.

In [ ]:
word2idx = ## create the word-to-index dictionary here

### Texto a vector, función

Ahora podemos escribir una función que convierta un texto a un vector de palabras. La función recibe un string de palabras y retorna un vector con el conteo de palabras. Un algoritmo genérico para hacer eso:

* Inicializa el vector de palabras con [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html), debería ser del largo del vocabulario.
* Separa el input string del texto en una lista de palabras con `.split(' ')`.
* Por cada palbra en esa lista, incrementa el elemento en el índice asociado a esa palabra, que lo obtienes de `word2idx`.

**Nota:** Como no todas las palabras que apareazcan están en el diccionario `vocab`, tendrás un error de key "index out of bounds", si se trata de acceder a una palabra que no esté. Puedes usar el método `.get` del diccionario `word2idx` para especificar el valor default de retorno cuando de un error. Por ejemplo, `word2idx.get(word, None)` retorna `None` si `word` no existe en el diccionario.

In [ ]:
def text_to_vector(text):
    
    pass

Si lo hiciste bien el siguiente código retornará:

```
text_to_vector('The tea is for a party to celebrate '
               'the movie so she has no time for a cake')[:65]
                   
array([0, 1, 0, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0])
```       

In [ ]:
text_to_vector('The tea is for a party to celebrate '
               'the movie so she has no time for a cake')[:65]

Ahora, itera en todas las reviews y conviertelas en un vector de palabras.

In [ ]:
word_vectors = np.zeros((len(reviews), len(vocab)), dtype=np.int_)
for ii, (_, text) in enumerate(reviews.iterrows()):
    word_vectors[ii] = text_to_vector(text[0])

In [ ]:
# Printing out the first 5 word vectors
word_vectors[:5, :23]

### Train, Validation, Test sets

Ahora que tenemos los vectores de palabras, estamos listos para dividir nuestros datos en train, validation y test sets. Utilizaremos para entrenar el train set, setearemos los hyperparameters con el validation set, y hasta el final evaluaremos el performace de la red neuronal con el test set. 
Aquí estamos usando la función `to_categorical` de TFLearn para remodelar los datos objetivo para que tengamos 2 unidades de output, para podamos clasificar utilizando una función de activación softmax. Nosotros no crearemos el set de validación eso lo hara TFLearn por nosotros.

In [ ]:
Y = (labels=='positive').astype(np.int_)
records = len(labels)

shuffle = np.arange(records)
np.random.shuffle(shuffle)
test_fraction = 0.9

train_split, test_split = shuffle[:int(records*test_fraction)], shuffle[int(records*test_fraction):]
trainX, trainY = word_vectors[train_split,:], to_categorical(Y.values[train_split], 2)
testX, testY = word_vectors[test_split,:], to_categorical(Y.values[test_split], 2)

In [ ]:
trainY

## Construyendo la Red Neuronal

[TFLearn](http://tflearn.org/) te permite construir la Red  [definiendo los layers](http://tflearn.org/layers/core/). 

### Input layer

Para el input layer, solo necesitamos decirle cuantas neuronas tendrá. Por ejemplo,
```
net = tflearn.input_data([None, 100])
```
esto creará una red con 100 neuronas en el input layer. El primer elemento de la lista, `None` en este caso, setea el tamaño del batch.

El número de inputs a la Red tiene que ser del tamaño de nuestra data, por ejemplo si tenemos 5000 palabras en nuestro vector de vocabulario entonces las neuronas de entrada deben ser 5000.


### Añadir layers

Para agregar nuevas 
To add new hidden layers, you use 

```
net = tflearn.fully_connected(net, n_units, activation='ReLU')
```

This adds a fully connected layer where every unit in the previous layer is connected to every unit in this layer. The first argument `net` is the network you created in the `tflearn.input_data` call. It's telling the network to use the output of the previous layer as the input to this layer. You can set the number of units in the layer with `n_units`, and set the activation function with the `activation` keyword. You can keep adding layers to your network by repeated calling `net = tflearn.fully_connected(net, n_units)`.

### Output layer

The last layer you add is used as the output layer. Therefore, you need to set the number of units to match the target data. In this case we are predicting two classes, positive or negative sentiment. You also need to set the activation function so it's appropriate for your model. Again, we're trying to predict if some input data belongs to one of two classes, so we should use softmax.

```
net = tflearn.fully_connected(net, 2, activation='softmax')
```

### Training
To set how you train the network, use 

```
net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss='categorical_crossentropy')
```

Again, this is passing in the network you've been building. The keywords: 

* `optimizer` sets the training method, here stochastic gradient descent
* `learning_rate` is the learning rate
* `loss` determines how the network error is calculated. In this example, with the categorical cross-entropy.

Finally you put all this together to create the model with `tflearn.DNN(net)`. So it ends up looking something like 

```
net = tflearn.input_data([None, 10])                          # Input
net = tflearn.fully_connected(net, 5, activation='ReLU')      # Hidden
net = tflearn.fully_connected(net, 2, activation='softmax')   # Output
net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss='categorical_crossentropy')
model = tflearn.DNN(net)
```

> **Exercise:** Below in the `build_model()` function, you'll put together the network using TFLearn. You get to choose how many layers to use, how many hidden units, etc.

In [ ]:
# Network building
def build_model():
    # This resets all parameters and variables, leave this here
    tf.reset_default_graph()
    
    #### Your code ####
    
    model = tflearn.DNN(net)
    return model

## Intializing the model

Next we need to call the `build_model()` function to actually build the model. In my solution I haven't included any arguments to the function, but you can add arguments so you can change parameters in the model if you want.

> **Note:** You might get a bunch of warnings here. TFLearn uses a lot of deprecated code in TensorFlow. Hopefully it gets updated to the new TensorFlow version soon.

In [ ]:
model = build_model()

## Training the network

Now that we've constructed the network, saved as the variable `model`, we can fit it to the data. Here we use the `model.fit` method. You pass in the training features `trainX` and the training targets `trainY`. Below I set `validation_set=0.1` which reserves 10% of the data set as the validation set. You can also set the batch size and number of epochs with the `batch_size` and `n_epoch` keywords, respectively. Below is the code to fit our the network to our word vectors.

You can rerun `model.fit` to train the network further if you think you can increase the validation accuracy. Remember, all hyperparameter adjustments must be done using the validation set. **Only use the test set after you're completely done training the network.**

In [ ]:
# Training
model.fit(trainX, trainY, validation_set=0.1, show_metric=True, batch_size=128, n_epoch=10)

## Testing

After you're satisified with your hyperparameters, you can run the network on the test set to measure its performance. Remember, *only do this after finalizing the hyperparameters*.

In [ ]:
predictions = (np.array(model.predict(testX))[:,0] >= 0.5).astype(np.int_)
test_accuracy = np.mean(predictions == testY[:,0], axis=0)
print("Test accuracy: ", test_accuracy)

## Try out your own text!

In [ ]:
# Helper function that uses your model to predict sentiment
def test_sentence(sentence):
    positive_prob = model.predict([text_to_vector(sentence.lower())])[0][1]
    print('Sentence: {}'.format(sentence))
    print('P(positive) = {:.3f} :'.format(positive_prob), 
          'Positive' if positive_prob > 0.5 else 'Negative')

In [ ]:
sentence = "Moonlight is by far the best movie of 2016."
test_sentence(sentence)

sentence = "It's amazing anyone could be talented enough to make something this spectacularly awful"
test_sentence(sentence)